<a href="https://colab.research.google.com/github/yukinaga/min_gen_agent_app/blob/main/section_2/01_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# チャットボットアプリ

In [ ]:
#================================================================
# 1. 必要なライブラリのインストール
#================================================================
# OpenAIのAPIを利用するためのライブラリと、
# Web UIを簡単に作成するためのGradioをインストールします。
# 先頭の `!` は、Colaboratory上でターミナルコマンドを実行するためのおまじないです。
!pip install openai gradio

#================================================================
# 2. ライブラリのインポートとAPIキーの設定
#================================================================
import openai  # OpenAIライブラリ
import gradio as gr  # Gradioライブラリ
from google.colab import userdata  # Colaboratoryのシークレットを読み込むためのライブラリ

# シークレットに保存したAPIキーを安全に読み込みます。
# 'OPENAI_API_KEY' は、先ほどシークレットに設定した「名前」と一致している必要があります。
try:
    openai.api_key = userdata.get('OPENAI_API_KEY')
    # OpenAIライブラリの新しいバージョン(v1.0.0以降)では、このようにクライアントを作成します。
    client = openai.OpenAI(api_key=openai.api_key)
except Exception as e:
    # APIキーの読み込みに失敗した場合にエラーメッセージを表示します。
    print("APIキーの読み込みに失敗しました。Colaboratoryのシークレット設定を確認してください。")
    print(f"エラー詳細: {e}")
    client = None # エラーが発生した場合はクライアントをNoneに設定

#================================================================
# 3. AIとの対話を行う関数を定義
#================================================================
# この関数がチャットボットの心臓部です。
# ユーザーからのメッセージ(message)と、今までの会話履歴(history)を受け取ります。
def chat_function(message, history):
    # APIキーが正しく設定されていない場合は、エラーメッセージを返して処理を中断します。
    if client is None:
        history.append([message, "エラー: OpenAIクライアントが初期化されていません。APIキーの設定を確認してください。"])
        # 入力欄をクリアし、エラーメッセージを表示した履歴を返します。
        return "", history

    # OpenAI APIが要求する形式に、今までの会話履歴を変換します。
    # Gradioの履歴は [[ユーザー1, AI1], [ユーザー2, AI2]] という形式です。
    # これを [{"role": "user", "content": ...}, {"role": "assistant", "content": ...}] の形式に直します。
    messages = []
    for user_msg, assistant_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": assistant_msg})

    # 最新のユーザーメッセージを会話履歴に追加します。
    messages.append({"role": "user", "content": message})

    try:
        # OpenAIのChatCompletion APIを呼び出します。
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # 使用するAIモデルを指定します。より高性能な "gpt-4" なども利用可能です。
            messages=messages,      # 変換した会話履歴を渡します。
            stream=True             # stream=Trueにすると、AIが文章を生成する過程をリアルタイムで受け取れます。
        )

        # AIからの返答を少しずつ結合していくための、空の文字列を用意します。
        bot_message = ""
        # stream形式で返ってくるレスポンスを一つずつ処理します。
        for chunk in response:
            # 返答の文章が含まれている場合のみ処理を続けます。
            if chunk.choices[0].delta.content is not None:
                # 返答の断片を追記していきます。
                bot_message += chunk.choices[0].delta.content
                # GradioのUIに2つの値を返します。
                # 1. 空文字列 "": 入力用テキストボックス(msg)に渡され、入力欄がクリアされます。
                # 2. 更新された会話履歴: チャット表示欄(chatbot)に渡され、AIの返信がリアルタイムで表示されます。
                yield "", history + [[message, bot_message]]

    except Exception as e:
        # API呼び出し中にエラーが発生した場合の処理
        print(f"API呼び出し中にエラーが発生しました: {e}")
        # ユーザーにエラーが発生したことを知らせます。
        yield "", history + [[message, f"エラーが発生しました: {e}"]]

#================================================================
# 4. Gradioを使ってWeb UIを作成・起動
#================================================================
# gr.Blocks() を使うと、コンポーネントを自由に配置してリッチなUIを構築できます。
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    # gr.Markdown() を使って、画面に説明文を表示します。#は大きな見出しを意味します。
    gr.Markdown("# シンプルAIチャットボット\nOpenAI APIとGradioを使って作成しました。")

    # gr.Chatbot() で、チャットの履歴を表示するコンポーネントを作成します。
    chatbot = gr.Chatbot(height=500, label="Chatbot")

    # gr.Textbox() で、ユーザーがメッセージを入力するテキストボックスを作成します。
    msg = gr.Textbox(label="あなたのメッセージ", placeholder="ここにメッセージを入力してEnterキーを押してください...")

    # gr.ClearButton() で、チャット履歴と入力欄をクリアするボタンを作成します。
    clear = gr.ClearButton([msg, chatbot], value="会話をリセット")

    # ユーザーがメッセージを送信したときの動作を定義します。
    # msg(テキストボックス)でEnterキーが押される(submit)と、以下の処理が実行されます。
    # 1. `chat_function` を呼び出す。
    # 2. `inputs`で指定された[msg, chatbot]の内容を、関数の引数(message, history)として渡す。
    # 3. `outputs`で指定された[msg, chatbot]に、関数の戻り値を設定してUIを更新する。
    msg.submit(chat_function, inputs=[msg, chatbot], outputs=[msg, chatbot])

# demo.launch() で、作成したUIを起動します。
# debug=True にすると、エラーが発生したときに詳細な情報を確認できるため開発に便利です。
demo.launch(debug=True)